### The code below are used for extracting information from excel order sheets
### File extension is .xlsx

### Please make sure all the information entered in the order sheets follow their correct format
### E.g. dates are in the format dd/mm/yyyy

In [1]:
'''Don't change the code in here'''

# import library 
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline 

### <font color='red'>Fill in details in the cell below as instructed</font>

In [2]:
# connect to Azure database
import pyodbc
server = '' # the server of your database
database = '' # the name of your database
username = '' # the user name of your database
password = '' # password for your user
driver= '{ODBC Driver 17 for SQL Server}' #don't change this

### This notebook lives in the same directory as the .xlsx files

### This function will find all the order froms in the current directory, extract the information and upload them to the database on Azure.
### This function should be run only once and may take some time depending on the number of .xlsx files in the folder.
###  Please make sure all excel files are closed while running this notebook

In [3]:
'''Don't change the code in here'''

path = os.getcwd() # get current directory
files = os.listdir(path)
files_xlsx = [f for f in files if f[-4:] == 'xlsx'] # list all .xlsx files in the directory 

# filter out the different kind of order forms: External, Internal, Roche 
external_list=[]
internal_list=[]
roche_list=[]

try:
    for i in files_xlsx:
        if i.find('External')!=-1:
            external_list.append(i)
        elif i.find('Internal')!=-1:
            internal_list.append(i)
        elif i.find('Roche')!=-1:
            roche_list.append(i)
        else:
            raise Exception
    print('external order form: ', external_list)
    print('internal order form: ', internal_list)
    print('Roche order form: ', roche_list)    
except Exception:
    print('Error, unexpected .xlsx file found in the folder! "{}"'.format(i))

    
# extract information from external order form
def extract_external(file_name_list):
    
    # get path of current directory
    path = os.getcwd()    
    # store information in list
    form = []
    #interate through the list of external forms
    for i in file_name_list:
        df = pd.read_excel(os.path.join(path, i))
        size = df.shape
        for j in range(15, size[0]): # iterate through multiple rows in one form
            row_single_order = [str(df.iat[4,1]), str(df.iat[2,1]), str(df.iat[j,0]), 
                                str(df.iat[3,1]), str(df.iat[j,3]), str(df.iat[j,2]), str(df.iat[j,1]), 
                                str(df.iat[j,4]), str(df.iat[1,1]), str(df.iat[6,1])]                       
            form.append(row_single_order) 
    return form


# extract information from internal order form
def extract_internal(file_name_list):
    
    # get path of current directory
    path = os.getcwd()    
    #init empty lists to hold information 
    form = []    
    #interate through the list of internal forms
    for i in file_name_list:
        df = pd.read_excel(os.path.join(path, i))
        size = df.shape        
        for j in range(9, size[0]):            
            if math.isnan(df.iat[j,9]) == False: # filter out empty rows
                row_single_order = [str(df.iat[1,2]), str(int(df.iat[0,6])), str(df.iat[j,1]), str(i[-13:-5]), 
                                    str(df.iat[j,3])+' '+str(df.iat[j,8]), str(df.iat[j,9]),
                                    str(df.iat[j,2]), 'NA', str(df.iat[0,2]), 'Internal']
                form.append(row_single_order) 
    return form


# extract information from Roche order form
def extract_roche(file_name_list):
    
    # get path of current directory
    path = os.getcwd()    
    #init empty lists to hold information 
    form = []    
    #interate through the list of Roche forms
    for i in file_name_list:
        df = pd.read_excel(os.path.join(path, i))
        size = df.shape        
        for j in range(11, size[0]):            
            if math.isnan(df.iat[j,2]) == False: # filter out empty rows
                row_single_order = [str(df.iat[4,1]), str(df.iat[2,1]), str(df.iat[j,0]), 
                                    str(df.iat[3,1]), str(df.iat[j,3]), str(df.iat[j,2]), str(df.iat[j,1]),
                                    str(df.iat[j,4]), str(df.iat[1,1]), 'Roche']
                form.append(row_single_order)
    return form

external order form:  ['External demo.xlsx']
internal order form:  []
Roche order form:  []


### The cell below to extract information

In [4]:
'''Don't change the code in here'''

# run the functions to extract the information 
all_update_values = extract_external(external_list) + extract_internal(internal_list) + extract_roche(roche_list)

# you can print and check the rows are correct before uploading to database
for i in all_update_values:
    print(i)

['3B29', '76239', '(Kits) K.KH.KHA. (Molecular Reagents) 41130', '2019-12-31 00:00:00', 'demo', '5', '6430112001', '0.2019', 'xy', 'nan']


### The cell below to upload extracted information to database
### <font color='red'>Please check</font>  the lists of order forms match expectation before running the cell below
### Upload to database is currently irreversible, however code could be implemented to allow deletion 

In [ ]:
'''Don't change the code in here'''

# upload this information to database by using cursor
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

# define insert query 
insert_query = '''INSERT INTO combined_date_formated_dataframe (deliverary_room, extension_no, 
                  order_NSV_category, order_date, order_description_and_pack_size, 
                  order_quantity, order_supplier_product_code, order_unit_cost, requester_names, supplier_name)
                  VALUES (?,?,?,?,?,?,?,?,?,?);'''
for row in all_update_values:
    values = tuple(row)
    cursor.execute(insert_query, values)

# commit the change
cnxn.commit()

# close the connection and remove the cursor
cursor.close()
cnxn.close()

### The cell below to export the database to csv

In [ ]:
'''Don't change the code in here'''

# export to csv (optional)
import sys
import csv

export_query='SELECT * FROM combined_date_formated_dataframe;'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute(export_query)
result=cursor.fetchall()
column_title = ('deliverary_room', 'extension_no', 
                'order_NSV_category', 'order_date', 'order_description_and_pack_size', 
                'order_quantity', 'order_supplier_product_code', 'order_unit_cost', 
                'requester_names', 'supplier_name')

### <font color='red'>Change</font> the file name here to aviod overwriting exsiting file

In [ ]:
file_name = '.csv' # file name must end in .csv
c = csv.writer(open(file_name, 'w', newline='')) 

In [ ]:
'''Don't change the code in here'''

c.writerow(column_title)
for x in result:
    c.writerow(x)

# close the connection and remove the cursor
cursor.close()
cnxn.close()

### You can check the updated database content in the csv file